## **4장 딥러닝 시작**

### 딥러닝 구조

#### 활성화 함수
- 렐루 함수와 소프트맥스 함수를 파이토치에서 구현하는 코드

In [ ]:
import torch

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)  # 은닉층
        self.relu = torch.nn.ReLU(inplace=True)
        self.out = torch.nn.Linear(n_hidden, n_output) # 출력층
        self.softmax = torch.nn.Softmax(dim=n_output)
    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)  # 은닉층을 위한 렐루 활성화 함수
        x = self.out(x)
        x = self.softmax(x)  # 출력층을 위한 소프트맥스 활성화 함수
        return x

#### 손실 함수

- `평균 제곱 오차` 파이토치에서 사용하기


```
import torch

loss_fn = torch.nn.MSELoss(reduction='sum')
y_pred = model(x)
loss = loss_fn(y_pred, y)
```



- `크로스 엔트로피 오차` 파이토치에서 사용하기


```
loss = nn.CrossEntropy()
input = torch.randn(5, 6, requires_grad=True)
# torch.randn: 평균이 0, 표준편차 1인 가우시안 정규분포 이용하여 숫자 생성
target = torch.empty(3, dtype=torch.long).random_(5)
# torch.empty: dtype torch.float32의 랜덤한 값으로 채워진 텐서 반환
output = loss(input, target)
output.backward()
```



#### 딥러닝의 문제점과 해결방안
- 과적합 문제 발생
- 해결법 : 드롭아웃



```
# 드롭아웃을 구현하는 예시코드
class DropoutModel(torch.nn.Module):
    def __init__(self):
        super(DropoutModel, self).__init__()
        self.layer1 = torch.nn.Linear(784, 1200)
        self.dropout1 = torch.nn.Dropout(0.5) # 50%의 노드를 무작위로 선택하여 사용하지 않겠다는 의미
        self.layer2 = torch.nn.Linear(1200, 1200)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.layer3 = torch.nn.Linear(1200, 10)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.dropout(x)
        x = F.relu(self.layer2(x))
        x = self.dropout2(x)
        return self.layer3(x)
```



- 성능 나빠지는 문제 발생
- 해결법 : 미니 배치 경사 하강법 & 확률적 경사 하강법


```
# 미니 배치 경사 하강법 구현하는 코드
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[1,2,3],[4,5,6],[7,8,9]]
        self.y_data = [[12],[18],[11]]
        def __len__(self):
            return len(self.x_data)
        def __getitem__(self, idx):
            x = torch.FloatTensor(self.x_data[idx])
            y = torch.FloatTensor(self.y_data[idx])
            return x, y
dataset = CustomDataset()
dataloader = DataLoader(
    dataset,  # 데이터셋
    batch_size = 2,  # 미니 배치 크기로 2의 제곱수 사용
    shuffle = True, # 데이터를 불러올 때마다 랜덤으로 섞어서 가져오기
)
```



#### 옵티마이저
- 확률적 경사 하강법의 변경 폭 불안정 문제 해결을 위해 옵티마이저 적용하기

- 속도를 조정하는 방법
    - 아다그라드(Adagrad)
```
    optmizer = torch.optim.Adagrad(model.parameters(), lr=0.01) # 학습률 기본값 1e-2
```

    - 아다델타(Adadelta)
```
    optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0) # 학습률 기본값 1.0
```
    - 암엠에스프롭(RMSProp)
```
    optimizer = torch.optim.RMSProp(model.parameters(), lr=0.01) # 학습률 기본값 1e-2
```

- 운동량을 조정하는 방법
    - 모멘텀(Momentum)
```
    optmizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    # momentum 값은 0.9에서 시작하여 0.95, 0.99 조금씩 증가시키며 사용.
```

    - 네스테로프 모멘텀(Nesterov Accelerated Gradient, NAG)
```
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
    # nesterov 기본값은 Flase
```

- 속도와 운동량에 대한 혼용 방법
    - 아담(Adam)
```
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # 학습률 기본값 1e-3
```